I used ChatGPT. The prompt I used was "Look at the code again. Self reflect and to critique and improve its summary. Do this in code"

In [ ]:
#!/usr/bin/env python3

from __future__ import annotations

import argparse
import json
import os
import random
import re
import tempfile
import textwrap
import time
from dataclasses import dataclass
from datetime import datetime, timezone
from email.utils import parseaddr
from enum import Enum
from pathlib import Path
from typing import Callable, Literal, Mapping, Tuple

# -----------------------------
# Defaults (can be overridden with --kb)
# -----------------------------
DEFAULT_KB: dict[str, str] = {
    "billing:refund_policy": (
        "Our standard refund window is 30 days from the charge date. "
        "If you're within that window and the service wasn't used, we can process a full refund."
    ),
    "billing:update_payment": (
        "To update your payment method, go to Account → Billing → Payment Methods and add a new card. "
        "Set it as default to use it next cycle."
    ),
    "technical:app_crash": (
        "Try these steps: 1) Force-quit the app, 2) Clear cache/storage, 3) Reinstall. "
        "If the crash persists, export logs from Settings → Advanced → Export Logs and share them with support."
    ),
    "technical:login_issue": (
        "Reset your password from the login screen. If 2FA is enabled, ensure your device time is automatic. "
        "VPNs/ad-blockers can interfere—try disabling temporarily."
    ),
    "order:track_package": (
        "Track your package from Account → Orders → 'Track'. Most carriers update statuses every 4–12 hours."
    ),
    "order:change_address": (
        "If the order hasn't shipped, change the address from Account → Orders. "
        "After shipment, we can request a carrier intercept (not guaranteed)."
    ),
}

# -----------------------------
# Types & Utilities
# -----------------------------
class Intent(str, Enum):
    BILLING = "billing"
    TECH = "technical"
    ORDER = "order"
    OTHER = "other"

class Status(str, Enum):
    NEW = "new"
    IN_PROGRESS = "in_progress"
    RESOLVED = "resolved"
    ESCALATED = "escalated"
    CLOSED = "closed"

@dataclass
class Message:
    ts: str
    role: Literal["agent", "user", "system"]
    text: str

class IO:
    """Console I/O abstraction for testability."""
    def __init__(self, quiet: bool = False):
        self.quiet = quiet

    def write(self, s: str):
        if not self.quiet:
            print(textwrap.fill(s, width=88))

    def read(self, prompt: str = "> ") -> str:
        try:
            return input(prompt).strip()
        except (EOFError, KeyboardInterrupt):
            return ""

class ScriptedIO(IO):
    """Feeds predefined inputs for demos/tests."""
    def __init__(self, lines: list[str], quiet: bool = False):
        super().__init__(quiet=quiet)
        self.lines = [ln.rstrip("\n") for ln in lines]
        self.i = 0

    def read(self, prompt: str = "> ") -> str:
        if self.i < len(self.lines):
            ans = self.lines[self.i]
            self.i += 1
            if not self.quiet:
                print(f"{prompt}{ans}")
            return ans.strip()
        return ""

WORD = re.compile(r"\b\w+\b")
EMAIL_RE = re.compile(r'[\w\.-]+@[\w\.-]+\.\w+')
ID_LONG_NUM_RE = re.compile(r"\b\d{10,}\b")  # long numeric IDs / phone-like sequences
STOPWORDS = {
    "the","a","an","and","or","to","for","of","in","on","at","is","it","this","that","with","my","your","our"
}

def utc_now() -> str:
    return datetime.now(timezone.utc).isoformat()

def redact_pii(s: str) -> str:
    s = EMAIL_RE.sub("[redacted-email]", s)
    s = ID_LONG_NUM_RE.sub("[redacted-id]", s)
    return s

def atomic_write_json(path: Path, obj) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    tmp = tempfile.NamedTemporaryFile('w', delete=False, dir=path.parent, encoding='utf-8')
    try:
        json.dump(obj, tmp, indent=2, ensure_ascii=False)
        tmp.flush(); os.fsync(tmp.fileno())
    finally:
        tmp.close()
    os.replace(tmp.name, path)

def valid_email(s: str) -> bool:
    return "@" in parseaddr(s)[1]

def _tokens_and_bigrams(text: str) -> Tuple[set[str], set[tuple[str,str]]]:
    toks = [m.group(0).lower() for m in WORD.finditer(text)]
    toks = [t for t in toks if t not in STOPWORDS]
    bigrams = set(zip(toks, toks[1:])) if len(toks) > 1 else set()
    return set(toks), bigrams

def detect_intent(text: str) -> tuple[Intent, float]:
    """
    Heuristic intent detection with token + bigram scoring.
    Uses phrases to avoid false positives (e.g., 'not working' vs tokens 'not'/'working').
    Returns (intent, confidence in [0,1]).
    """
    tokens, bigrams = _tokens_and_bigrams(text)

    # keyword sets (no 'not'/'working' as singletons)
    bill_kw = {"charge","refund","invoice","payment","bill","subscription","price"}
    tech_kw = {"error","bug","issue","crash","login","slow","cant","cannot","timeout"}
    order_kw = {"order","shipping","delivery","package","address","tracking"}

    # phrase weights (bigrams)
    tech_phr = {("not","working"), ("cant","login"), ("cannot","login"), ("app","crash"), ("log","in")}

    scores = {
        Intent.BILLING: len(tokens & bill_kw),
        Intent.TECH:    len(tokens & tech_kw) + sum(1 for bg in bigrams if bg in tech_phr) * 2,  # phrases weigh more
        Intent.ORDER:   len(tokens & order_kw),
    }

    best = max(scores, key=scores.get)
    best_score = scores[best]
    second_best = max(v for k,v in scores.items() if k != best) if len(scores) > 1 else 0
    total = sum(scores.values())

    # confidence calibrated: margin over best+second, prefer higher separation
    margin = best_score - second_best
    denom = best_score + second_best if (best_score + second_best) > 0 else 1
    conf = max(0.0, min(1.0, margin / denom))

    if best_score == 0 or conf < 0.5:
        return Intent.OTHER, conf
    return best, conf

def new_ticket_id(rng: random.Random) -> str:
    # seconds + ns for extra entropy; rng for repeatability if seeded
    return f"TCK-{int(time.time())}-{time.time_ns() % 1000:03d}-{rng.randint(100,999)}"

def wrap(s: str) -> str:
    return textwrap.fill(s, width=88)

# -----------------------------
# Core Simulator
# -----------------------------
Transition = Callable[[], Status]

class CustomerServiceSim:
    def __init__(
        self,
        io: IO | None = None,
        rng: random.Random | None = None,
        out_dir: Path | None = None,
        kb: Mapping[str, str] | None = None,
    ):
        self.io = io or IO()
        self.rng = rng or random.Random()
        self.out_dir = out_dir or Path.cwd()
        self.kb: Mapping[str, str] = kb or DEFAULT_KB

        self.transcript: list[Message] = []
        self.customer = {"name": None, "email": None}
        self.intent: Intent = Intent.OTHER
        self.intent_conf: float = 0.0
        self.ticket_id: str | None = None
        self.status: Status = Status.NEW

        # metrics
        self.resolutions = 0
        self.escalations = 0
        self.csat_scores: list[int] = []

    # --- I/O helpers that also log to transcript ---
    def say(self, role: Literal["agent","system"], text: str):
        self.io.write(text)
        self.transcript.append(Message(ts=utc_now(), role=role, text=text))

    def hear(self, text: str):
        self.transcript.append(Message(ts=utc_now(), role="user", text=text))

    def ask(self, prompt: str) -> str:
        self.io.write(prompt)
        ans = self.io.read("> ")
        self.hear(ans)
        return ans

    def choose(self, prompt: str, options: list[str]) -> str:
        lines = [prompt] + [f"  {i}. {opt}" for i, opt in enumerate(options, 1)]
        self.io.write("\n".join(lines))
        while True:
            sel = self.io.read("> ")
            if sel.isdigit() and 1 <= int(sel) <= len(options):
                return options[int(sel) - 1]
            self.io.write("Please enter a valid number from the list.")

    # --- Flow steps ---
    def capture_customer(self):
        self.say("agent", "Hi! Thanks for contacting support. I’m Ava. What’s your name?")
        name = self.io.read("> ") or "Customer"
        self.hear(name)
        self.customer["name"] = name

        self.say("agent", f"Nice to meet you, {name}! What’s the best email to reach you?")
        while True:
            email = self.io.read("> ")
            self.hear(email)
            if not email or valid_email(email):
                self.customer["email"] = email or ""
                break
            self.say("agent", "Hmm, that email doesn’t look right. Try again or type 'skip'.")
            if email.lower() == "skip":
                self.customer["email"] = ""
                break

    def capture_issue(self):
        issue = self.ask("Briefly describe what you need help with today:")
        self.intent, self.intent_conf = detect_intent(issue)
        if self.intent == Intent.OTHER or self.intent_conf < 0.5:
            self.say("agent", "Thanks! I’m not fully sure which area this is. Could you pick one?")
            choice = self.choose(
                "Pick the closest area:",
                ["Billing", "Technical", "Order", "Other"]
            )
            mapping = {
                "Billing": Intent.BILLING, "Technical": Intent.TECH,
                "Order": Intent.ORDER, "Other": Intent.OTHER
            }
            self.intent = mapping.get(choice, Intent.OTHER)
        self.say("agent", f"Got it—routing you to {self.intent.value} support.")

    # --- Branch handlers ---
    def handle_billing(self) -> Status:
        choice = self.choose(
            "Billing topics I can help with right now:",
            ["Refund policy", "Update payment method", "Something else"]
        )
        if choice == "Refund policy":
            self.say("agent", self.kb.get("billing:refund_policy", "Refund info unavailable."))
            return Status.RESOLVED
        elif choice == "Update payment method":
            self.say("agent", self.kb.get("billing:update_payment", "Payment update info unavailable."))
            return Status.RESOLVED
        else:
            notes = self.ask("Tell me more about the billing issue:")
            _ = notes  # included in transcript
            self.say("agent", "Thanks—I may need a billing specialist to review this.")
            return Status.ESCALATED

    def handle_technical(self) -> Status:
        choice = self.choose(
            "Technical topics I can help with:",
            ["App is crashing", "Login trouble", "Something else"]
        )
        if choice == "App is crashing":
            self.say("agent", self.kb.get("technical:app_crash", "Crash steps unavailable."))
            worked = self.choose("Did that help?", ["Yes, resolved", "No, still broken"])
            return Status.RESOLVED if worked == "Yes, resolved" else Status.ESCALATED
        elif choice == "Login trouble":
            self.say("agent", self.kb.get("technical:login_issue", "Login steps unavailable."))
            worked = self.choose("Did that help?", ["Yes, resolved", "No, still blocked"])
            return Status.RESOLVED if worked == "Yes, resolved" else Status.ESCALATED
        else:
            details = self.ask("Describe the issue and any error messages:")
            _ = details
            self.say("agent", "Thanks—I’ll route this to an engineer.")
            return Status.ESCALATED

    def handle_order(self) -> Status:
        choice = self.choose(
            "Order topics I can help with:",
            ["Track a package", "Change shipping address", "Something else"]
        )
        if choice == "Track a package":
            order_id = self.ask("What’s your order number?")
            _ = order_id
            self.say("agent", self.kb.get("order:track_package", "Tracking info unavailable."))
            return Status.RESOLVED
        elif choice == "Change shipping address":
            stage = self.choose("Has the order shipped yet?", ["Not shipped", "Already shipped / unknown"])
            if stage == "Not shipped":
                self.say("agent", self.kb.get("order:change_address", "Address change info unavailable."))
                return Status.RESOLVED
            else:
                self.say("agent", "I’ll request a carrier intercept, but it’s not guaranteed.")
                return Status.ESCALATED
        else:
            notes = self.ask("Tell me what you need help with on the order:")
            _ = notes
            self.say("agent", "Thanks—this may require a fulfillment specialist.")
            return Status.ESCALATED

    def handle_other(self) -> Status:
        notes = self.ask("Thanks! Could you share a few details?")
        _ = notes
        self.say("agent", "I’ll find the right person to help.")
        return Status.ESCALATED

    def branch_flow(self) -> Status:
        self.status = Status.IN_PROGRESS
        table: dict[Intent, Transition] = {
            Intent.BILLING: self.handle_billing,
            Intent.TECH:    self.handle_technical,
            Intent.ORDER:   self.handle_order,
            Intent.OTHER:   self.handle_other,
        }
        handler = table.get(self.intent, self.handle_other)
        return handler()

    def escalate(self):
        self.ticket_id = new_ticket_id(self.rng)
        self.say(
            "agent",
            f"I’m escalating this to a specialist. Your ticket number is {self.ticket_id}. "
            "You’ll hear from us by email soon."
        )
        self.status = Status.ESCALATED
        self.escalations += 1

    def csat(self):
        score = self.choose("Before you go, how satisfied are you with today’s support?", ["1","2","3","4","5"])
        try:
            n = int(score)
        except ValueError:
            n = 3
        self.csat_scores.append(n)
        comment = self.ask("Any quick feedback for us? (optional)")
        _ = comment
        self.say("agent", "Thank you! Your feedback helps us improve.")

    def save_transcript(self):
        tid = self.ticket_id or new_ticket_id(self.rng)
        record = {
            "ticket_id": tid,
            "customer": {
                "name": redact_pii(self.customer.get("name") or ""),
                "email": redact_pii(self.customer.get("email") or ""),
            },
            "intent": self.intent.value,
            "intent_confidence": round(self.intent_conf, 3),
            "status": self.status.value,
            "created_utc": utc_now(),
            "metrics": {
                "resolutions": self.resolutions,
                "escalations": self.escalations,
                "avg_csat": round(sum(self.csat_scores)/len(self.csat_scores), 2) if self.csat_scores else None
            },
            "transcript": [
                {"ts": m.ts, "role": m.role, "text": redact_pii(m.text)} for m in self.transcript
            ],
        }
        filename = self.out_dir / f"transcript_{tid}.json"
        try:
            atomic_write_json(filename, record)
            self.say("system", f"Transcript saved to {filename}")
        except Exception as e:
            # Best-effort logging; don't crash silently
            self.say("system", f"Failed to save transcript to {filename}: {e!r}")

    def run(self):
        try:
            self.capture_customer()
            self.capture_issue()
            outcome = self.branch_flow()
            if outcome == Status.RESOLVED:
                self.status = Status.RESOLVED
                self.resolutions += 1
                self.say("agent", "Happy to help! I’m marking this as resolved. 🎉")
            else:
                self.escalate()
            self.csat()
            if self.status == Status.RESOLVED:
                self.status = Status.CLOSED
        except Exception as e:
            # Log the error into transcript for transparency; still save transcript.
            self.status = Status.ESCALATED
            self.say("system", f"Unexpected error: {e!r}")
            raise
        finally:
            self.save_transcript()

# -----------------------------
# CLI
# -----------------------------
def parse_args():
    p = argparse.ArgumentParser(description="Customer Service Flow Simulator (improved)")
    p.add_argument("--out", type=Path, default=Path.cwd(), help="Output directory for transcripts")
    p.add_argument("--seed", type=int, help="Random seed for deterministic ticket IDs")
    p.add_argument("--scripted", type=Path, help="Path to newline-separated user replies")
    p.add_argument("--kb", type=Path, help="Path to a JSON knowledge base to override defaults")
    p.add_argument("--quiet", action="store_true", help="Suppress console output (useful for tests/CI)")
    return p.parse_args()

def load_kb(path: Path | None) -> tuple[dict[str, str], str | None]:
    if not path:
        return DEFAULT_KB, None
    try:
        return json.loads(path.read_text(encoding="utf-8")), None
    except Exception as e:
        return DEFAULT_KB, f"KB override failed ({e!r}); using defaults."

def main():
    args = parse_args()
    rng = random.Random(args.seed) if args.seed is not None else random.Random()
    kb, kb_err = load_kb(args.kb)
    if args.scripted and args.scripted.exists():
        lines = args.scripted.read_text(encoding="utf-8").splitlines()
        io: IO = ScriptedIO(lines, quiet=args.quiet)
    else:
        io = IO(quiet=args.quiet)

    sim = CustomerServiceSim(io=io, rng=rng, out_dir=args.out, kb=kb)
    if kb_err:
        sim.say("system", kb_err)

    sim.run()

if __name__ == "__main__":
    main()
